In [5]:
import getpass
import os
import torch
import numpy as np
from PIL import Image
import torchvision
from torch.autograd import Variable
from torch.utils.data import Dataset
from torchvision import transforms
import pandas as pd
import pydicom
import matplotlib.pyplot as plt
import torch.nn.functional as F
from collections import OrderedDict
#from skimage import transform
import sys
from sklearn.metrics import roc_curve, auc
import copy
from sklearn.metrics import f1_score
import torchvision.models as models
import re
import xml.etree.ElementTree as ET
import cv2
from matplotlib import rcParams, animation, rc
from ipywidgets import interact, interactive, fixed
from ipywidgets.widgets import *

#from display import read_image, draw_boxes, draw_grid, draw_text
#import display

In [6]:
# with this function you set the value of the environment variable CUDA_VISIBLE_DEVICES
# to set which GPU to use
# it also reserves this amount of memory for your exclusive use. This might be important for 
# not having other people using the resources you need in shared systems
# the homework was tested in a GPU with 4GB of memory, and running this function will require at least
# as much
# if you want to test in a GPU with less memory, you can call this function
# with the argument minimum_memory_mb specifying how much memory from the GPU you want to reserve
def define_gpu_to_use(minimum_memory_mb = 3800):
    gpu_to_use = None
    try: 
        os.environ['CUDA_VISIBLE_DEVICES']
        print('GPU already assigned before: ' + str(os.environ['CUDA_VISIBLE_DEVICES']))
        return
    except:
        pass
    torch.cuda.empty_cache()
    for i in range(16):
        free_memory = !nvidia-smi --query-gpu=memory.free -i $i --format=csv,nounits,noheader
        if free_memory[0] == 'No devices were found':
            break
        free_memory = int(free_memory[0])
        if free_memory>minimum_memory_mb-500:
            gpu_to_use = i
            break
    if gpu_to_use is None:
        print('Could not find any GPU available with the required free memory of ' +str(minimum_memory_mb) + 'MB. Please use a different system for this assignment.')
    else:
        os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_to_use)
        print('Chosen GPU: ' + str(gpu_to_use))
        x = torch.rand((256,1024,minimum_memory_mb-500)).cuda()
        x = torch.rand((1,1)).cuda()
        del x
define_gpu_to_use()

Chosen GPU: 0


In [7]:
def load_raw_images(root,split):
    set_of_transforms = {}
    set_of_transforms['train'] = transforms.Compose(
        [transforms.RandomRotation(30),
         transforms.RandomResizedCrop(224),
         transforms.RandomHorizontalFlip(),
        transforms.ToTensor(), 
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])])
    set_of_transforms['test'] = transforms.Compose(
        [transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])])
    set_of_transforms['validate'] = set_of_transforms['test']
    

    whole_dataset = torchvision.datasets.ImageFolder(root=root+'/'+split, transform=set_of_transforms[split])
    
    return whole_dataset   

def load_grayscale_images(root,split):
    set_of_transforms = {}
    set_of_transforms['train'] = transforms.Compose(
        [transforms.RandomRotation(30),
         transforms.RandomResizedCrop(224),
         transforms.RandomHorizontalFlip(),
         transforms.Grayscale(num_output_channels=3),
         transforms.ToTensor(), 
         transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])])
    set_of_transforms['test'] = transforms.Compose(
        [transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.Grayscale(num_output_channels=3),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])])
    set_of_transforms['validate'] = set_of_transforms['test']
    

    whole_dataset = torchvision.datasets.ImageFolder(root=root+'/'+split, transform=set_of_transforms[split])
    
    return whole_dataset   

def find_classes(dir):
    classes = [d for d in os.listdir(dir) if os.path.isdir(os.path.join(dir, d))]
    classes.sort()
    class_to_idx = {classes[i]: i for i in range(len(classes))}
    return classes, class_to_idx

#color images
path_to_img_folder = '../color/60_20_20'
train_data = load_raw_images(path_to_img_folder,'train')
test_data = load_raw_images(path_to_img_folder,'test')
val_data = load_raw_images(path_to_img_folder,'validate')

#grayscale images
train_grayscale_data = load_grayscale_images(path_to_img_folder,'train')
test_grayscale_data = load_grayscale_images(path_to_img_folder,'test')
val_grayscale_data = load_grayscale_images(path_to_img_folder,'validate')

#segmented images
path_to_segment_img_folder = '../segment/60_20_20segmented'
train_segmented_data = load_raw_images(path_to_segment_img_folder,'train')
test_segmented_data = load_raw_images(path_to_segment_img_folder,'test')
val_segmented_data = load_raw_images(path_to_segment_img_folder,'validate')


label_names,label_indices = find_classes('../color/60_20_20/train')
label_names_dict = dict((v,k) for k,v in label_indices.items())
print ("Length of the train set:{}".format(len(train_data)))
print ("Length of the test set:{}".format(len(test_data)))
print ("Length of the validation set:{}".format(len(val_data)))
print (" ")
print ("The label names along with its corresponding key:{}".format(label_names_dict))

Length of the train set:16051
Length of the test set:6453
Length of the validation set:4264
 
The label names along with its corresponding key:{0: 'Apple_Frogeye_Spot', 1: 'Apple___Apple_scab', 2: 'Apple___Cedar_apple_rust', 3: 'Apple___healthy', 4: 'Blueberry___healthy', 5: 'Cherry_including_sour___Powdery_mildew', 6: 'Cherry_including_sour___healthy', 7: 'Corn_maize___Cercospora_leaf_spot Gray_leaf_spot', 8: 'Corn_maize___Common_rust_', 9: 'Corn_maize___Northern_Leaf_Blight', 10: 'Corn_maize___healthy', 11: 'Grape___Black_rot', 12: 'Grape___Esca_Black_Measles', 13: 'Grape___Leaf_blight_Isariopsis_Leaf_Spot', 14: 'Grape___healthy', 15: 'Orange___Haunglongbing_Citrus_greening', 16: 'Peach___Bacterial_spot', 17: 'Peach___healthy', 18: 'Pepper_bell___Bacterial_spot', 19: 'Pepper_bell___healthy', 20: 'Potato___Early_blight', 21: 'Potato___Late_blight', 22: 'Potato___healthy', 23: 'Raspberry___healthy', 24: 'Soybean___healthy', 25: 'Squash___Powdery_mildew', 26: 'Strawberry___Leaf_scorch',

In [8]:
#original images (RGB) data loader
train_loader = torch.utils.data.DataLoader(train_data, shuffle = True, batch_size = 84, num_workers = 3)
test_loader = torch.utils.data.DataLoader(test_data, shuffle = True, batch_size = 84, num_workers = 3)
val_loader = torch.utils.data.DataLoader(val_data, shuffle = True, batch_size = 84, num_workers = 3)

#grayscale data loader
train_grayscale_loader = torch.utils.data.DataLoader(train_grayscale_data, shuffle = True, batch_size = 84, num_workers = 3)
test_grayscale_loader = torch.utils.data.DataLoader(test_grayscale_data, shuffle = True, batch_size = 84, num_workers = 3)
val_grayscale_loader = torch.utils.data.DataLoader(val_grayscale_data, shuffle = True, batch_size = 84, num_workers = 3)

#segmented images data loader
train_segmented_loader = torch.utils.data.DataLoader(train_segmented_data, shuffle = True, batch_size = 84, num_workers = 3)
test_segmented_loader = torch.utils.data.DataLoader(test_segmented_data, shuffle = True, batch_size = 84, num_workers = 3)
val_segmented_loader = torch.utils.data.DataLoader(val_segmented_data, shuffle = True, batch_size = 84, num_workers = 3)

#visualization
train_loader_vis = torch.utils.data.DataLoader(train_data, shuffle = True, batch_size = 1, num_workers = 3)

print ("Number of batches train set:{}".format(len(train_loader)))
print ("Number of batches test set:{}".format(len(test_loader)))
print ("Number of batches validation set:{}".format(len(val_loader)))

Number of batches train set:192
Number of batches test set:77
Number of batches validation set:51


In [65]:
#densenet169
classification_model = models.densenet169(pretrained=True)
for param in classification_model.parameters():
    param.requires_grad = False
print (classification_model)

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplac

In [66]:
classifier=torch.nn.Sequential(OrderedDict([('fc1', torch.nn.Linear(1664, 512)),
                           ('relu', torch.nn.ReLU()), 
                           ('dropout', torch.nn.Dropout(p=0.337)),
                           ('fc2', torch.nn.Linear(512, 38)),
                           ('output', torch.nn.LogSoftmax(dim=1))
                             ]))
classification_model.classifier = classifier

In [67]:
def train_classification_model(data_loader, model, criterion, optimizer, lr_decay, num_epochs,mode='train'):
    for epoch in range(num_epochs):
        model.train()
        loss_sum = []; acc = 0
        for i in data_loader:
            input_img =i[0].cuda()
            target = i[1].cuda() 
            if mode=='train':
                optimizer.zero_grad()
            predicted_output = model(input_img)
            loss = criterion(predicted_output, target)
            _,pred_out = torch.max(predicted_output,1)    
            if mode=='train':
                loss.backward()
                optimizer.step()
            loss_sum.append(loss.item())
            acc+=torch.sum(pred_out==target.data)
        epoch_loss = sum(loss_sum)/len(loss_sum)
        epoch_accuracy = acc.item()/len(train_data)
        print ("epoch:"+str(epoch))
        print ("Loss:{:.6f}".format(epoch_loss))
        print ("Accuracy:{:.6f}".format(epoch_accuracy))
        print (" ")



In [68]:
#test models
def test_model(model,data_loader):
    model.eval()
    accuracy_list=[]
    with torch.no_grad():
        predicted_output_concat = np.zeros([84])
        target_concat = np.zeros([84])
        for i in data_loader:
            input_img=i[0].cuda()
            #print (input_img[0].shape)
            target=i[1]
            target=target.cuda()
            pred_output = model(input_img)
            eq = (target.data == torch.exp(pred_output).max(1)[1])
            pred_out = torch.exp(pred_output).max(1)[1]
            accuracy_list.append(eq.type_as(torch.FloatTensor()).mean())
            predicted_output_concat = np.concatenate((predicted_output_concat, pred_out.cpu().detach().numpy()), axis = 0)
            target_concat = np.concatenate((target_concat, target.cpu().detach().numpy()), axis = 0)
        print ("Test accuracy:{:.6f}".format(sum(accuracy_list)/len(accuracy_list)))
        #print (target_concat)
        #print (target_concat.shape)
        #print (predicted_output_concat)
        #print (predicted_output_concat.shape)
    print ("Macro F1 score:{}".format(f1_score(target_concat, predicted_output_concat, average="macro")))
    print ("Micro F1 score:{}".format(f1_score(target_concat, predicted_output_concat, average="micro")))


## Adam + 0.001

### DenseNet accuracies on color datatset

In [10]:
#train
criterion = torch.nn.NLLLoss()
classification_model = classification_model.cuda()
optimizer = torch.optim.Adam(classification_model.classifier.parameters(), lr=0.001)
#lr_schedule = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
lr_schedule = 0 
train_classification_model(train_loader, classification_model, criterion, optimizer,lr_schedule,10,'train')

epoch:0
Loss:1.370612
Accuracy:0.641268
 
epoch:1
Loss:0.623804
Accuracy:0.816772
 
epoch:2
Loss:0.511978
Accuracy:0.847922
 
epoch:3
Loss:0.449003
Accuracy:0.863560
 
epoch:4
Loss:0.417524
Accuracy:0.870600
 
epoch:5
Loss:0.402888
Accuracy:0.875023
 
epoch:6
Loss:0.398371
Accuracy:0.876207
 
epoch:7
Loss:0.377754
Accuracy:0.881627
 
epoch:8
Loss:0.375874
Accuracy:0.884306
 
epoch:9
Loss:0.351190
Accuracy:0.891533
 


#### Validation Accuracy

In [12]:
test_model(classification_model, val_loader)

Test accuracy:0.908132
[ 0.  0.  0. ... 21.  5. 28.]
(4348,)
[ 0.  0.  0. ... 21.  5. 28.]
(4348,)
Macro F1 score:0.8734251246469528
Micro F1 score:0.9098436062557498


#### Test Accuracy

In [13]:
test_model(classification_model, test_loader)

Test accuracy:0.918509
[ 0.  0.  0. ... 15. 34. 10.]
(6537,)
[ 0.  0.  0. ... 15. 34. 10.]
(6537,)
Macro F1 score:0.8851078312353295
Micro F1 score:0.9195349548722656


### DenseNet accuracies on segmented dataset

In [19]:
#train
criterion = torch.nn.NLLLoss()
classification_model = classification_model.cuda()
optimizer = torch.optim.Adam(classification_model.classifier.parameters(), lr=0.001)
#lr_schedule = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
lr_schedule = 0 
train_classification_model(train_segmented_loader, classification_model, criterion, optimizer,lr_schedule,10,'train')

epoch:0
Loss:1.378010
Accuracy:0.630615
 
epoch:1
Loss:0.678285
Accuracy:0.796025
 
epoch:2
Loss:0.566608
Accuracy:0.826553
 
epoch:3
Loss:0.512933
Accuracy:0.838577
 
epoch:4
Loss:0.486442
Accuracy:0.844994
 
epoch:5
Loss:0.450192
Accuracy:0.858015
 
epoch:6
Loss:0.466206
Accuracy:0.852408
 
epoch:7
Loss:0.437082
Accuracy:0.861815
 
epoch:8
Loss:0.420607
Accuracy:0.864245
 
epoch:9
Loss:0.422610
Accuracy:0.867796
 


#### Validation accuracy

In [20]:
test_model(classification_model, val_segmented_loader)

Test accuracy:0.903783
[ 0.  0.  0. ... 35. 28. 24.]
(4346,)
[ 0.  0.  0. ... 35. 28. 24.]
(4346,)
Macro F1 score:0.8683163237904613
Micro F1 score:0.9058904739990796


#### Test accuracy

In [21]:
test_model(classification_model, test_segmented_loader)

Test accuracy:0.907669
[ 0.  0.  0. ... 35. 12.  8.]
(6530,)
[ 0.  0.  0. ... 35. 12.  8.]
(6530,)
Macro F1 score:0.8778762779203025
Micro F1 score:0.9090352220520673


### DenseNet accuracies on grayscale dataset

In [34]:
#train
criterion = torch.nn.NLLLoss()
classification_model = classification_model.cuda()
optimizer = torch.optim.Adam(classification_model.classifier.parameters(), lr=0.001)
#lr_schedule = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
lr_schedule = 0 
train_classification_model(train_grayscale_loader, classification_model, criterion, optimizer,lr_schedule,10,'train')

epoch:0
Loss:1.654747
Accuracy:0.554919
 
epoch:1
Loss:0.979705
Accuracy:0.707121
 
epoch:2
Loss:0.840886
Accuracy:0.742321
 
epoch:3
Loss:0.786080
Accuracy:0.756090
 
epoch:4
Loss:0.739952
Accuracy:0.769547
 
epoch:5
Loss:0.705836
Accuracy:0.779951
 
epoch:6
Loss:0.690023
Accuracy:0.783814
 
epoch:7
Loss:0.685005
Accuracy:0.788611
 
epoch:8
Loss:0.691969
Accuracy:0.783689
 
epoch:9
Loss:0.670668
Accuracy:0.793159
 


#### Validation Accuracy

In [36]:
test_model(classification_model, val_grayscale_loader)

Test accuracy:0.813944
[ 0.  0.  0. ... 18. 32. 24.]
(4348,)
[ 0.  0.  0. ... 18. 32. 27.]
(4348,)
Macro F1 score:0.7345969795978357
Micro F1 score:0.8176172953081877


#### Test Accuracy

In [35]:
test_model(classification_model, test_grayscale_loader)

Test accuracy:0.830026
[ 0.  0.  0. ... 37. 37. 24.]
(6537,)
[ 0.  0.  0. ... 37. 37. 24.]
(6537,)
Macro F1 score:0.7540847679787847
Micro F1 score:0.8321860180510938


## Adam + 0.0001

### DenseNet accuracies on colored dataset

In [13]:
#train
criterion = torch.nn.NLLLoss()
classification_model = classification_model.cuda()
optimizer = torch.optim.Adam(classification_model.classifier.parameters(), lr=0.0001)
#lr_schedule = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
lr_schedule = 0 
train_classification_model(train_loader, classification_model, criterion, optimizer,lr_schedule,10,'train')

epoch:0
Loss:2.633979
Accuracy:0.367454
 
epoch:1
Loss:1.579117
Accuracy:0.641082
 
epoch:2
Loss:1.135039
Accuracy:0.729986
 
epoch:3
Loss:0.906096
Accuracy:0.781758
 
epoch:4
Loss:0.787531
Accuracy:0.799078
 
epoch:5
Loss:0.692816
Accuracy:0.822067
 
epoch:6
Loss:0.632378
Accuracy:0.833406
 
epoch:7
Loss:0.582675
Accuracy:0.845306
 
epoch:8
Loss:0.531078
Accuracy:0.857143
 
epoch:9
Loss:0.515038
Accuracy:0.857641
 


#### Validation Accuracy

In [14]:
test_model(classification_model, val_loader)

Test accuracy:0.915470
Macro F1 score:0.8716233409760225
Micro F1 score:0.9174333026678932


#### Test Accuracy

In [15]:
test_model(classification_model, test_loader)

Test accuracy:0.922562
Macro F1 score:0.8811526792147862
Micro F1 score:0.9235123145173627


### DenseNet accuracies on segmented dataset

In [20]:
#train
criterion = torch.nn.NLLLoss()
classification_model = classification_model.cuda()
optimizer = torch.optim.Adam(classification_model.classifier.parameters(), lr=0.0001)
#lr_schedule = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
lr_schedule = 0 
train_classification_model(train_segmented_loader, classification_model, criterion, optimizer,lr_schedule,10,'train')

epoch:0
Loss:2.604974
Accuracy:0.359479
 
epoch:1
Loss:1.579855
Accuracy:0.621706
 
epoch:2
Loss:1.158403
Accuracy:0.713663
 
epoch:3
Loss:0.944101
Accuracy:0.756464
 
epoch:4
Loss:0.823808
Accuracy:0.777272
 
epoch:5
Loss:0.731480
Accuracy:0.802068
 
epoch:6
Loss:0.680121
Accuracy:0.814217
 
epoch:7
Loss:0.627965
Accuracy:0.825681
 
epoch:8
Loss:0.585611
Accuracy:0.836272
 
epoch:9
Loss:0.567634
Accuracy:0.838826
 


#### Validation Accuracy

In [21]:
test_model(classification_model, val_segmented_loader)

Test accuracy:0.887225
Macro F1 score:0.8344310634582476
Micro F1 score:0.8893235158766684


#### Test Accuracy

In [22]:
test_model(classification_model, test_segmented_loader)

Test accuracy:0.890717
Macro F1 score:0.8562902324180538
Micro F1 score:0.8921898928024502


### DenseNet accuracies on grayscale dataset

In [27]:
#train
criterion = torch.nn.NLLLoss()
classification_model = classification_model.cuda()
optimizer = torch.optim.Adam(classification_model.classifier.parameters(), lr=0.0001)
#lr_schedule = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
lr_schedule = 0 
train_classification_model(train_grayscale_loader, classification_model, criterion, optimizer,lr_schedule,10,'train')

epoch:0
Loss:2.764966
Accuracy:0.321101
 
epoch:1
Loss:1.869660
Accuracy:0.542334
 
epoch:2
Loss:1.458904
Accuracy:0.623575
 
epoch:3
Loss:1.237538
Accuracy:0.677777
 
epoch:4
Loss:1.113610
Accuracy:0.699209
 
epoch:5
Loss:1.028021
Accuracy:0.715532
 
epoch:6
Loss:0.953009
Accuracy:0.732602
 
epoch:7
Loss:0.906718
Accuracy:0.744315
 
epoch:8
Loss:0.870253
Accuracy:0.749112
 
epoch:9
Loss:0.824943
Accuracy:0.762071
 


#### Validation Accuracy

In [28]:
test_model(classification_model, val_grayscale_loader)

Test accuracy:0.799384
Macro F1 score:0.7157554019447123
Micro F1 score:0.8031278748850046


/home/u1136693/env_dir/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


#### Test Accuracy

In [29]:
test_model(classification_model, test_grayscale_loader)

Test accuracy:0.808227
Macro F1 score:0.7313127316161322
Micro F1 score:0.8106164907449901


## SGD + Momentum and 0.001

### DenseNet Accuracies on colored dataset

In [34]:
#train
criterion = torch.nn.NLLLoss()
classification_model = classification_model.cuda()
optimizer = torch.optim.SGD(classification_model.classifier.parameters(), lr=0.001, momentum=0.9)
#lr_schedule = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
lr_schedule = 0 
train_classification_model(train_loader, classification_model, criterion, optimizer,lr_schedule,10,'train')

epoch:0
Loss:3.103117
Accuracy:0.234378
 
epoch:1
Loss:2.361066
Accuracy:0.422030
 
epoch:2
Loss:1.839527
Accuracy:0.574357
 
epoch:3
Loss:1.500082
Accuracy:0.646128
 
epoch:4
Loss:1.270327
Accuracy:0.692729
 
epoch:5
Loss:1.111358
Accuracy:0.725811
 
epoch:6
Loss:1.002574
Accuracy:0.750109
 
epoch:7
Loss:0.912586
Accuracy:0.766058
 
epoch:8
Loss:0.843601
Accuracy:0.782568
 
epoch:9
Loss:0.783495
Accuracy:0.797957
 


#### Validation Accuracy

In [35]:
test_model(classification_model, val_loader)

Test accuracy:0.869325
Macro F1 score:0.7657349925026674
Micro F1 score:0.8718951241950322


#### Test Accuracy

In [36]:
test_model(classification_model, test_loader)

Test accuracy:0.879158
Macro F1 score:0.779430599809091
Micro F1 score:0.8808321860180511


### DenseNet Accuracies on segemented dataset

In [41]:
#train
criterion = torch.nn.NLLLoss()
classification_model = classification_model.cuda()
optimizer = torch.optim.SGD(classification_model.classifier.parameters(), lr=0.001, momentum=0.9)
#lr_schedule = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
lr_schedule = 0 
train_classification_model(train_segmented_loader, classification_model, criterion, optimizer,lr_schedule,10,'train')

epoch:0
Loss:3.077225
Accuracy:0.234378
 
epoch:1
Loss:2.341298
Accuracy:0.425020
 
epoch:2
Loss:1.832699
Accuracy:0.568314
 
epoch:3
Loss:1.514857
Accuracy:0.627562
 
epoch:4
Loss:1.294201
Accuracy:0.673914
 
epoch:5
Loss:1.139066
Accuracy:0.710610
 
epoch:6
Loss:1.032650
Accuracy:0.726746
 
epoch:7
Loss:0.945635
Accuracy:0.750234
 
epoch:8
Loss:0.886963
Accuracy:0.762071
 
epoch:9
Loss:0.829973
Accuracy:0.775528
 


#### Validation Accuracy

In [42]:
test_model(classification_model, val_segmented_loader)

Test accuracy:0.849944
Macro F1 score:0.75830165326387
Micro F1 score:0.8527381500230097


#### Test Accuracy

In [43]:
test_model(classification_model, test_segmented_loader)

Test accuracy:0.848799
Macro F1 score:0.7628943711002876
Micro F1 score:0.8509954058192956


### DenseNet Accuracies on grayscale dataset

In [48]:
#train
criterion = torch.nn.NLLLoss()
classification_model = classification_model.cuda()
optimizer = torch.optim.SGD(classification_model.classifier.parameters(), lr=0.001, momentum=0.9)
#lr_schedule = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
lr_schedule = 0 
train_classification_model(train_grayscale_loader, classification_model, criterion, optimizer,lr_schedule,10,'train')

epoch:0
Loss:3.126634
Accuracy:0.231761
 
epoch:1
Loss:2.499142
Accuracy:0.381409
 
epoch:2
Loss:2.073859
Accuracy:0.493676
 
epoch:3
Loss:1.771320
Accuracy:0.559591
 
epoch:4
Loss:1.577489
Accuracy:0.595851
 
epoch:5
Loss:1.410192
Accuracy:0.628247
 
epoch:6
Loss:1.307244
Accuracy:0.651548
 
epoch:7
Loss:1.218548
Accuracy:0.670488
 
epoch:8
Loss:1.153140
Accuracy:0.677715
 
epoch:9
Loss:1.097252
Accuracy:0.696156
 


#### Validation Accuracy

In [49]:
test_model(classification_model, val_grayscale_loader)

Test accuracy:0.751036
Macro F1 score:0.6233183181398452
Micro F1 score:0.7562097516099356


#### Test Accuracy

In [50]:
test_model(classification_model, test_grayscale_loader)

Test accuracy:0.762402
Macro F1 score:0.6433805887978354
Micro F1 score:0.7656417316811993


## SGD + Momentum and 0.0001

### DenseNet Accuracies on colored dataset

In [55]:
#train
criterion = torch.nn.NLLLoss()
classification_model = classification_model.cuda()
optimizer = torch.optim.SGD(classification_model.classifier.parameters(), lr=0.0001, momentum=0.9)
#lr_schedule = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
lr_schedule = 0 
train_classification_model(train_loader, classification_model, criterion, optimizer,lr_schedule,10,'train')

epoch:0
Loss:3.537405
Accuracy:0.108965
 
epoch:1
Loss:3.337838
Accuracy:0.197246
 
epoch:2
Loss:3.223786
Accuracy:0.233817
 
epoch:3
Loss:3.136199
Accuracy:0.252445
 
epoch:4
Loss:3.058843
Accuracy:0.265529
 
epoch:5
Loss:2.972534
Accuracy:0.274625
 
epoch:6
Loss:2.896929
Accuracy:0.287521
 
epoch:7
Loss:2.808199
Accuracy:0.302100
 
epoch:8
Loss:2.727082
Accuracy:0.321787
 
epoch:9
Loss:2.650814
Accuracy:0.343032
 


#### Validation Accuracy

In [56]:
test_model(classification_model, val_loader)

Test accuracy:0.371659
Macro F1 score:0.1414947772089892
Micro F1 score:0.38385464581416745


#### Test Accuracy

In [57]:
test_model(classification_model, test_loader)

Test accuracy:0.380811
Macro F1 score:0.14379367822766265
Micro F1 score:0.38871041762276276


### DenseNet Accuracies on segmented dataset

In [62]:
#train
criterion = torch.nn.NLLLoss()
classification_model = classification_model.cuda()
optimizer = torch.optim.SGD(classification_model.classifier.parameters(), lr=0.0001, momentum=0.9)
#lr_schedule = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
lr_schedule = 0 
train_classification_model(train_segmented_loader, classification_model, criterion, optimizer,lr_schedule,10,'train')

epoch:0
Loss:3.539915
Accuracy:0.104479
 
epoch:1
Loss:3.331700
Accuracy:0.193633
 
epoch:2
Loss:3.218904
Accuracy:0.227151
 
epoch:3
Loss:3.135442
Accuracy:0.251511
 
epoch:4
Loss:3.056415
Accuracy:0.265342
 
epoch:5
Loss:2.975466
Accuracy:0.272693
 
epoch:6
Loss:2.893177
Accuracy:0.284780
 
epoch:7
Loss:2.813249
Accuracy:0.294935
 
epoch:8
Loss:2.735249
Accuracy:0.306523
 
epoch:9
Loss:2.656679
Accuracy:0.331319
 


#### Validation Accuracy

In [63]:
test_model(classification_model, val_segmented_loader)

Test accuracy:0.336157
Macro F1 score:0.10078145675938845
Micro F1 score:0.3490566037735849


#### Test Accuracy

In [64]:
test_model(classification_model, test_segmented_loader)

Test accuracy:0.343353
Macro F1 score:0.10025174688745901
Micro F1 score:0.35176110260336907


### DenseNet Accuracies on grayscale dataset

In [69]:
#train
criterion = torch.nn.NLLLoss()
classification_model = classification_model.cuda()
optimizer = torch.optim.SGD(classification_model.classifier.parameters(), lr=0.0001, momentum=0.9)
#lr_schedule = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
lr_schedule = 0 
train_classification_model(train_grayscale_loader, classification_model, criterion, optimizer,lr_schedule,10,'train')

epoch:0
Loss:3.524429
Accuracy:0.092705
 
epoch:1
Loss:3.341487
Accuracy:0.150707
 
epoch:2
Loss:3.256544
Accuracy:0.201981
 
epoch:3
Loss:3.183970
Accuracy:0.233506
 
epoch:4
Loss:3.122438
Accuracy:0.250950
 
epoch:5
Loss:3.058310
Accuracy:0.261791
 
epoch:6
Loss:2.992608
Accuracy:0.269827
 
epoch:7
Loss:2.926996
Accuracy:0.277677
 
epoch:8
Loss:2.855185
Accuracy:0.287521
 
epoch:9
Loss:2.794666
Accuracy:0.300978
 


#### Validation Accuracy

In [70]:
test_model(classification_model, val_grayscale_loader)

Test accuracy:0.324113
Macro F1 score:0.11529917761094383
Micro F1 score:0.33693652253909845


#### Test Accuracy

In [71]:
test_model(classification_model, test_grayscale_loader)

Test accuracy:0.329677
Macro F1 score:0.11409441986511411
Micro F1 score:0.3383815205751874
